http://large.stanford.edu/courses/2007/ph210/pelc2/
https://dsp.stackexchange.com/questions/13142/digital-distortion-effect-algorithm

In [1]:
import os
import numpy as np
import tensorflow as tf
import glob
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
wave_num=5
sz=256
def createGraph():
  d=tf.constant(0.23)
  b=tf.constant(0.03)
  fs=tf.constant(44100.0)
  gam=tf.constant(1.7)
  rnd_f=tf.random_uniform([1], 110, 880)[0]
  bpm=tf.constant(80.0)
  nharm=tf.floor(tf.div(fs,(2.0*rnd_f)))
  Nbeats = tf.constant(8)
  T=60.0/bpm
  
  nharm_i=tf.cast(nharm, dtype=tf.int32)
  An=tf.zeros([1,nharm_i], dtype=tf.float32)
  
  js=tf.range(1.0,nharm,1.0, dtype=tf.float32)
  tf_pi=tf.constant(np.pi, dtype=tf.float32)

  An=tf.sin(js*np.pi*d) * 2.0/(tf.pow(tf_pi,2.0) * tf.pow(js,2.0) * d * (1-d))
  dfn=tf.sqrt(1.0+js*js*b*b)

  Nt=tf.cast(fs*T, dtype=tf.int32)
  sj=tf.zeros([Nt])
  ns = tf.range(tf.cast(Nt, dtype=tf.float32), dtype=tf.float32)

  ns = tf.reshape(ns, [-1,1])
  js=tf.reshape(js, [1,-1])
  dfn=tf.reshape(dfn, [1,-1])
  sj=An*tf.exp(-gam*js*ns/fs) * tf.sin(2.0*tf_pi*ns*rnd_f*(dfn*js)/fs)
  wave=tf.reduce_sum(sj,axis=1)
  
  rnd_phase=tf.random_uniform([1], 0, Nt-sz-1, dtype=tf.int32)[0]

  return ns[rnd_phase:rnd_phase+sz], wave[rnd_phase:rnd_phase+sz]

def outputWave(path):
  with tf.Session() as sess:
    
    g=createGraph()
    for i in np.arange(wave_num):
      t, waves=sess.run(g)

      data = np.zeros([2,sz], dtype=np.float32)
      data[0] = waves[:sz]
      data[1] = distortion(waves[:sz])
      fn = "{0}/{1:0>4}".format(path, i) + ".bin"
      data.tofile(fn)

def distortion(x):
    return np.sign(x)*(1-np.exp(-np.abs(x**2)))

#with tf.Session() as sess:
#    g=createGraph()
#    t,x = sess.run(g)
#plt.plot(t,x)

In [3]:
out_dir="./data"
os.makedirs(out_dir, exist_ok=True)
outputWave(out_dir)

In [4]:
input_paths = glob.glob(os.path.join(out_dir, "*.bin"))
input_paths.sort()

filename_queue = tf.train.string_input_producer(input_paths)
reader = tf.WholeFileReader()
key, value = reader.read(filename_queue)
image = tf.decode_raw(value, tf.float32)
image = tf.reshape(image,[2,1,tf.constant(sz),1])

sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

out=sess.run(image)

coord.request_stop()
coord.join(threads)

xs = range(sz)
plt.plot(xs, out[0,0,:,0], xs, out[1,0,:,0])